____
**Universidad Tecnológica Nacional, Buenos Aires**<br/>
**Ingeniería Industrial**<br/>
**Cátedra de Ciencia de Datos - Curso I5521 - Turno jueves noche**<br/>
**Trabajo Práctico Final:** EDA : Analisis Exploratorio de los Datos<br/>
**Elaborado por:** Cristian D. Andrades & Joaquin Campos Guevara.
____

La plataforma Airbnb les pidió ayuda para predecir los precios de los hospedajes para algunas ciudades dentro de USA. Para ellos se les presentará un dataset con un listado de 19.309 publicaciones con 29  variables que muestran algunas características de las propiedades.

Partes prácticas del trabajo a desarrollar:

*   Desarrollar en python un EDA (un análisis exploratorio sobre de el dataset en cuestión)
*   Desarrollar en python un pipeline de Machine Learning para predecir la variable Price. Aplicar también algún método de la reducción de la dimensionalidad visto en clase y volver a predecir el problema en cuestión.
*   Redactar reporte técnico describiendo el trabajo.




In [ ]:
# Importamos las librerías para manipular datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# el dataset que vamos a manipular esta en nuestro googledrive, con lo cual tenemos que montar la unidad
from google.colab import drive
drive.mount('/content/drive')
# ahora sí guardamos el dataset en una variable
airbnb_df = pd.read_csv('/content/drive/MyDrive/TP_Final_Ciencia_de_Datos_2025/airbnb_us.csv')

# Importamos librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import VarianceThreshold



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# imprimimos las primeras 5 filas del dataframe para entender mejor como son los datos a manipular
airbnb_df.head()

,id,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,...,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,price
0,13418779,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,...,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0,750.0
1,3808709,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,...,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0,115.0
2,12422935,Apartment,Private room,"{TV,""Wireless Internet"",Heating,""Smoke detecto...",2,1.0,Real Bed,strict,True,SF,...,-122.429526,Comfort Suite San Francisco,Noe Valley,3,100.0,https://a0.muscache.com/im/pictures/82509143-4...,94131,1.0,1.0,85.0
3,180792,House,Private room,"{TV,""Cable TV"",""Wireless Internet"",""Pets live ...",2,1.0,Real Bed,moderate,True,SF,...,-122.501095,Cozy Garden Studio - Private Entry,Richmond District,159,99.0,https://a0.muscache.com/im/pictures/0ed6c128-7...,94121,1.0,1.0,120.0
4,2658946,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",6,1.5,Real Bed,strict,True,DC,...,-77.031189,Charming 2 bdrm in trendy U/14th streets w/par...,U Street Corridor,13,89.0,NaN,20009,2.0,3.0,200.0


In [ ]:
# usamos la funcion .info() para entender el tipo de dato de cada variable, las dimensiones del dataframe y cuántos valores no nulos hay por variable.
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19309 entries, 0 to 19308
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      19309 non-null  int64  
 1   property_type           19309 non-null  object 
 2   room_type               19309 non-null  object 
 3   amenities               19309 non-null  object 
 4   accommodates            19309 non-null  int64  
 5   bathrooms               19274 non-null  float64
 6   bed_type                19309 non-null  object 
 7   cancellation_policy     19309 non-null  object 
 8   cleaning_fee            19309 non-null  bool   
 9   city                    19309 non-null  object 
 10  description             19309 non-null  object 
 11  first_review            15355 non-null  object 
 12  host_has_profile_pic    19306 non-null  object 
 13  host_identity_verified  19306 non-null  object 
 14  host_response_rate      15013 non-null

In [ ]:
# vemos que hay variables que contienen valores nulos y son irrelevantes para nuestro objetivo.
# para eivtar problemas más adelante y que el codigo corra más holgado, listamos las columnas a eliminar del DF
variables_a_eliminar = [
    'id',
    'name',
    'description',
    'thumbnail_url',
    'host_has_profile_pic',
    'host_identity_verified',
    'host_response_rate',
    'host_since',
    'first_review',
    'review_scores_rating',
    'last_review',
    'zipcode',
    'amenities',
]
# eliminamos estas columnas y modificamos el DF en una misma linea
airbnb_df.drop(variables_a_eliminar, axis=1, inplace=True)

In [ ]:
# analizamos nuevamente el df luego de esta ultima modificacion y vemos que aun tenemos valores nulos en algunas columnas que no queremos eliminar
airbnb_df.isnull().sum()

,0
property_type,0
room_type,0
accommodates,0
bathrooms,35
bed_type,0
cancellation_policy,0
cleaning_fee,0
city,0
instant_bookable,0
latitude,0


In [ ]:
# para la dimension neighbourhood, tenemos 1458 registros nulos de 19.308, es decir el 7,5% de los datos.
# decidimos conservar los registros, completando los nulos en 'neighbourhood' con el valor más repetido para la correspondiente 'city':
# para lograrlo, agrupamos por ciudad y obtenemos el neighbourhood más frecuente en esa city con una funcion lambda (si hubira un empate, .iloc[0] elige el primer neighbourhood)
moda_city = (
    airbnb_df.groupby('city')['neighbourhood'].agg(lambda x: x.mode().iloc[0])
)
# ahora creamos un diccionario con indice city y sus neighbourhood más frecuentes. luego completaremos los nulos recorriendo este diccionario
moda_dict = moda_city.to_dict()
# ahora recorremos la columna neighbourhood. Si encuentra un nulo, remplaza con el diccionario. si no es nulo, deja el valor original
airbnb_df['neighbourhood'] = airbnb_df.apply(
    lambda row: moda_dict[row['city']] if pd.isna(row['neighbourhood']) else row['neighbourhood'],
    axis=1
)

# al ser pocos registros y todas variables numericas, vamos a completar el dataframe con la mediana de cada variable
for col in ['bathrooms','bedrooms','beds']:
  airbnb_df[col] = airbnb_df[col].fillna(airbnb_df[col].median())

In [ ]:
# aun hay que seguir tranformando el dataframe
# la columna instant_bookable debería ser un booleano donde f=False y t=True
airbnb_df['instant_bookable'].head()

,instant_bookable
0,f
1,t
2,t
3,f
4,t


In [ ]:
# para evitar problemas, vamos a modificar el tipo de esta columna y tambien a declarar algunas variables como tipo 'string' ya que python no está pudiendo identificarlas correctamente y las cataloga como 'object'.
airbnb_df[['property_type','room_type','bed_type','cancellation_policy','city', 'neighbourhood']]=airbnb_df[['property_type','room_type','bed_type','cancellation_policy','city', 'neighbourhood']].astype('string')
airbnb_df['instant_bookable']=airbnb_df['instant_bookable'].map({'f': False , 't': True})
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19309 entries, 0 to 19308
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   property_type        19309 non-null  string 
 1   room_type            19309 non-null  string 
 2   accommodates         19309 non-null  int64  
 3   bathrooms            19309 non-null  float64
 4   bed_type             19309 non-null  string 
 5   cancellation_policy  19309 non-null  string 
 6   cleaning_fee         19309 non-null  bool   
 7   city                 19309 non-null  string 
 8   instant_bookable     19309 non-null  bool   
 9   latitude             19309 non-null  float64
 10  longitude            19309 non-null  float64
 11  neighbourhood        19309 non-null  string 
 12  number_of_reviews    19309 non-null  int64  
 13  bedrooms             19309 non-null  float64
 14  beds                 19309 non-null  float64
 15  price                19309 non-null 

In [ ]:
# para las variables categoricas relevantes para el posterior analisis, habría que generar variables dummies numericas para poder considerar estas dimensiones en el modelo de machine learning
city_dummies = pd.get_dummies(airbnb_df['city'])
property_dummies= pd.get_dummies(airbnb_df['property_type'])
room_dummies= pd.get_dummies(airbnb_df['room_type'])
bed_dummies= pd.get_dummies(airbnb_df['bed_type'])
cancellation_dummies= pd.get_dummies(airbnb_df['cancellation_policy'])
neigh_dummies = pd.get_dummies(airbnb_df['neighbourhood'])
# ahora las unimos al DF a trabajar
airbnb_df = pd.concat([airbnb_df, city_dummies, property_dummies, room_dummies, bed_dummies, cancellation_dummies, neigh_dummies], axis=1)
# falta eliminar las columnas categoricas originales del DF para que solo queden las dummies
airbnb_df = airbnb_df.drop(
    ['city', 'property_type', 'room_type', 'bed_type', 'cancellation_policy', 'neighbourhood'], axis=1)

In [ ]:
# ahora procedemos a dividir el dataframe entre variables independientes 'X' y dependientes 'Y'
y = airbnb_df['price']
x = airbnb_df.drop(columns=['price'])

In [ ]:
# usamos la librería de sklearn y determinamos el tamaño del conjunto de testeo en 20%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
# primero ejecutaremos sin reduccion de dimensionalidad en el pipeline1 o 'PL1'
# al finalizar, haremos otra prediccion en el pipeline2 o 'PL2' esta vez reduciendo la dimensionalidad
# usaremos la funcion 'Pipline' de sklearn para ordenar la ejecucion del codigo

PL1 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
# una vez escalados los datos y habiendo definido el modelo, lo entrenamos
PL1.fit(x_train, y_train)
# obtenemos las predicciones del modelo
y_pred_PL1 = PL1.predict(x_test)
# el modelo PL1 tiene el siguiente score:
mse_PL1 = mean_squared_error(y_test, y_pred_PL1)
rmse_PL1 = np.sqrt(mse_PL1)
r2_PL1 = r2_score(y_test, y_pred_PL1)
print(f"PL1  RMSE= {rmse_PL1:.2f} -- R2: {r2_PL1:.3f}")


PL1  RMSE= 163.74 -- R2: 0.355


In [ ]:
# para el PL2 vamos a probar otro modelo más complejo con la esperanza de que mejore el score del PL1
# al tener tantas dummies para las var categoricas, la dimensionalidad del df es elevada y seguramente requiere una reduccion.

In [ ]:
# ahora ejecutaremos el PL2 utilizando la tecnica de Gradient Boosting con reduccion de dimensionalidad PCA
# en el scaler usamos with_mean=False por las variables dummies que explican las variables categoricas
# antes del entrenamiento usamos threshold 0.1 para eliminar las variables casi nulas que no son relevantes. esto va a agilizar el proceso
# para la reduccion fijamos los componentes a conciderar en 100 para capturar una buena porcion de la varianza de la variable 'price'
# fijamos el nuemro de arboles en 100 con n_estimators y una complejidad de 3 niveles con max_deph
PL2 = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('var_thresh', VarianceThreshold(threshold=0.01)),
    ('reduccion', PCA(n_components=100)),
    ('modelo', GradientBoostingRegressor(random_state=0))
])
param_grid = {
    "modelo__n_estimators": [100],
    "modelo__learning_rate": [0.05, 0.1],
    "modelo__max_depth": [3]
}

grid_PL2 = GridSearchCV(
    PL2,
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
grid_PL2.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('var_thresh',
                                        VarianceThreshold(threshold=0.01)),
                                       ('reduccion', PCA(n_components=100)),
                                       ('modelo',
                                        GradientBoostingRegressor(random_state=0))]),
             n_jobs=-1,
             param_grid={'modelo__learning_rate': [0.05, 0.1],
                         'modelo__max_depth': [3],
                         'modelo__n_estimators': [100]},
             scoring='neg_root_mean_squared_error')

In [ ]:

print("Mejores parámetros encontrado con el modelo de PL2:", grid_PL2.best_params_)
# guardamos en una variable el mejor modelo resultado del gridsearch en PL2
PL2_optimo = grid_PL2.best_estimator_
# obtenemos las predicciones del modelo PL2_optimo, es decir entrenado con los mejores parametros
y_pred_PL2 = PL2_optimo.predict(x_test)
# el modelo PL2 tiene el siguiente score:
mse_PL2 = mean_squared_error(y_test, y_pred_PL2)
rmse_PL2 = np.sqrt(mse_PL2)
r2_PL2 = r2_score(y_test, y_pred_PL2)
print(f"PL2  RMSE= {rmse_PL2:.2f} -- R2: {r2_PL2:.3f}")

Mejores parámetros encontrado con el modelo de PL2: {'modelo__learning_rate': 0.1, 'modelo__max_depth': 3, 'modelo__n_estimators': 100}
PL2  RMSE= 160.95 -- R2: 0.376


In [ ]:
print(f"PL1 modelo de Regresion Lineal      -> RMSE= {rmse_PL1:.2f} -- R2= {r2_PL1:.3f}")
print(f"PL2 modelo Gradient Boosting + PCA  -> RMSE= {rmse_PL2:.2f} -- R2= {r2_PL2:.3f}")

PL1 modelo de Regresion Lineal      -> RMSE= 163.74 -- R2= 0.355
PL2 modelo Gradient Boosting + PCA  -> RMSE= 160.95 -- R2= 0.376


Informe Tecnico:
Luego del analisis exploratorio de los datos, descartamos algunas dimensiones y confeccionamos 2 modelos distintos para intentar predecir el precio de los Airbnb por ciudad dentro de nuestro data frame.
El modelo que mejor score obtuvo fue el ejecutado en el Pipeline 2 (modelo Gradient Boosting con reduccion de variables) y su coeficiente de determinacion fue tan solo R2= 37,6% es decir que logra explicar el 37,6% de la varianza de los precios de los Airbnb por ciudad de nuestro DF.
Es posible que alguna variable categorica no está aportando al modelo y lo este sobreajustando. Intentamos escalar logaritmicamente la variable objetivo, lo cual tendría una distribucion más simetrica y aun así el score no mejoró. También probamos otros modelos más complejos y demandantes como el SVR con PCA y aun así no pudimos mejorar el coeficiente de determinacion R2.
Cocluímos que el data set no tiene todas las variables que pueden llegar a explicar la varianza en el precio de estos inmuebles.